<a href="https://colab.research.google.com/github/Ape108/Quantum_Portfolio_Optimization/blob/main/MVP_Walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Overview: QAOA for Portfolio Optimization**

This document details the workflow from the `MVP_Completed.ipynb` notebook, which maps a financial optimization problem to a quantum circuit using the Quantum Approximate Optimization Algorithm (QAOA).

---

### **1. Financial Modeling Inputs**

To begin, we derive key financial metrics from historical price data.

* **Expected Annual Return ($\mu$)**
    * **Purpose**: To quantify the expected profit for each asset.
    * **Formula**:
        $$\mu = \text{mean}(R_{\text{minute}}) \times (\text{MIN\_PER\_DAY} \times \text{TRADING\_DAYS})$$
    * **Logic**: The mean of short-term returns is annualized to project long-term performance.

* **Annualized Covariance Matrix ($\Sigma$)**
    * **Purpose**: To quantify the risk. The diagonal represents individual asset volatility, and the off-diagonal represents how assets move together.
    * **Formula**:
        $$\Sigma = \text{cov}(R_{\text{minute}}) \times (\text{MIN\_PER\_DAY} \times \text{TRADING\_DAYS})$$
    * **Logic**: Provides the mathematical basis for calculating the total risk of a given portfolio.

---

### **2. The QUBO Formulation**

The problem is framed as a Quadratic Unconstrained Binary Optimization (QUBO), where the goal is to find a binary vector $x$ that minimizes an objective function.

* **Objective Function**:
    $$\min_{x \in \{0,1\}^n} \left( x^T \Sigma x - q \mu^T x \right) + P \left( \sum_{i=1}^n x_i - K \right)^2$$

* **Components**:
    * **Risk Term ($x^T \Sigma x$)**: Calculates the portfolio's total variance (risk). The goal is to minimize this.
    * **Return Term ($-q \mu^T x$)**: Represents the portfolio's expected return. It is negated to align with the minimization framework. The parameter $q$ balances risk vs. return.
    * **Constraint Penalty ($P (\sum x_i - K)^2$)**: Enforces the rule to select exactly `K` assets by adding a large penalty cost to any solution that violates this rule.

---

### **3. Mapping to a Quantum Problem (QUBO to Ising)**

QAOA is designed to solve problems in the Ising model format, so the QUBO must be converted.

* **Transformation**: The binary variables $x_i \in \{0, 1\}$ are mapped to spin variables $z_i \in \{-1, +1\}$.
  $$x_i = \frac{1 - z_i}{2}$$

* **Logic**: This conversion reframes the optimization problem into a physics problem: finding the ground-state energy of an Ising Hamiltonian ($H$). This is a task well-suited for QAOA and is performed by the `qubo_to_ising_sparsepauli` function.

---

### **4. The QAOA Logic**

QAOA is a hybrid algorithm that uses both classical and quantum resources to find approximate solutions.

* **The Ansatz Circuit**: The algorithm builds a parameterized quantum circuit (`Ansatz`) from repeating layers of two operators:
    1.  **Cost Hamiltonian ($e^{-i\gamma H_C}$)**: Applies a phase to each state based on its cost, as defined by the Ising Hamiltonian.
    2.  **Mixer Hamiltonian ($e^{-i\beta H_M}$)**: Enables transitions between different possible solution states.

* **The Role of Gamma ($\gamma$) and Beta ($\beta$)**: These are the variational parameters that the classical optimizer tunes.
    * **Gamma ($\gamma$)**: This parameter controls the application of the Cost Hamiltonian. It determines how strongly the phases related to the problem's cost are applied. Think of it as the "exploitation" phase, where the algorithm refines and reinforces better-scoring solutions based on the problem's objective function.
    * **Beta ($\beta$)**: This parameter controls the application of the Mixer Hamiltonian. It drives the transitions between all possible solution states (e.g., flipping qubits). Think of it as the "exploration" phase, where the algorithm moves around the solution space to find new potential answers.

* **The Hybrid Optimization Loop**:
    * A classical optimizer (`L-BFGS-B`) chooses circuit parameters ($\gamma, \beta$).
    * The quantum circuit is executed with these parameters to produce a state.
    * The energy of this state is measured.
    * The classical optimizer uses this energy to choose better parameters, repeating the process until it finds the parameters that minimize the energy.

* **The `reps` Parameter**: This defines the number of Cost/Mixer layers in the circuit. For `reps=p`, there will be `p` pairs of $(\gamma, \beta)$ to optimize.

---

### **5. Extracting the Solution**

The final optimized parameters $(\vec{\gamma}^*, \vec{\beta}^*)$ are used to run the circuit one last time.

* **Logic**: The final quantum state is a superposition where the basis state corresponding to the optimal solution (lowest energy) has the highest amplitude.
* **Extraction**: By simulating measurements, we find the bitstring with the highest probability. This bitstring represents the optimal asset selection, with a '1' in position $i$ indicating that asset $i$ is included in the portfolio.